In [1]:
from line_solver import *
import numpy as np
GlobalConstants.setVerbose(VerboseLevel.STD)

In [2]:
#Series of Fork-Join Networks#Multiple fork-join stages in series
model = Network('model')

In [3]:
# Create series of fork-join stages
source = Source(model, 'Source') #First fork-join stage
fork1 = Fork(model, 'Fork1')
queue1a = Queue(model, 'Queue1A', SchedStrategy.PS)

queue1b = Queue(model, 'Queue1B', SchedStrategy.PS)
join1 = Join(model, 'Join1', fork1) #Second fork-join stage
fork2 = Fork(model, 'Fork2')
queue2a = Queue(model, 'Queue2A', SchedStrategy.FCFS)

queue2b = Queue(model, 'Queue2B', SchedStrategy.FCFS)
join2 = Join(model, 'Join2', fork2)

sink = Sink(model, 'Sink')
jobclass = OpenClass(model, 'Class1')

In [4]:
#Service configurations
source.setArrival(jobclass, Exp(0.4))
queue1a.setService(jobclass, Exp(1.2))
queue1b.setService(jobclass, Exp(1.5))
queue2a.setService(jobclass, Exp(1.8))
queue2b.setService(jobclass, Exp(1.0))

In [5]:
# Series fork-join routing
P = model.initRoutingMatrix() #First stage
P.set(jobclass, jobclass, source, fork1, 1.0)
P.set(jobclass, jobclass, fork1, queue1a, 1.0)
P.set(jobclass, jobclass, fork1, queue1b, 1.0)
P.set(jobclass, jobclass, queue1a, join1, 1.0)
P.set(jobclass, jobclass, queue1b, join1, 1.0) #Between stages
P.set(jobclass, jobclass, join1, fork2, 1.0) #Second stage
P.set(jobclass, jobclass, fork2, queue2a, 1.0)
P.set(jobclass, jobclass, fork2, queue2b, 1.0)
P.set(jobclass, jobclass, queue2a, join2, 1.0)
P.set(jobclass, jobclass, queue2b, join2, 1.0) #To sink
P.set(jobclass, jobclass, join2, sink, 1.0)
model.link(P)

In [6]:
# Solve series system
solver = SolverJMT(model, 'samples', 15000)
print(solver.getAvgTable())

JMT Model: /tmp/workspace/jsim/14820943968983302100/jmodel.jsim
   Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Source   Class1  0.0000  0.0000  0.0000  0.0000  0.0000  0.4000
1  Queue1A   Class1  0.4804  0.3361  1.2683  0.6342  0.4000  0.3999
2  Queue1B   Class1  0.3686  0.2697  0.8925  0.4462  0.4000  0.3999
3    Join1   Class1  0.4451  0.0000  0.5516  0.5516  0.8097  0.3999
4  Queue2A   Class1  0.2935  0.2250  0.7251  0.3626  0.3999  0.3999
5  Queue2B   Class1  0.6899  0.4031  1.6644  0.8322  0.3999  0.4008
6    Join2   Class1  0.5463  0.0000  0.6751  0.6751  0.8022  0.3985
   Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Source   Class1  0.0000  0.0000  0.0000  0.0000  0.0000  0.4000
1  Queue1A   Class1  0.4804  0.3361  1.2683  0.6342  0.4000  0.3999
2  Queue1B   Class1  0.3686  0.2697  0.8925  0.4462  0.4000  0.3999
3    Join1   Class1  0.4451  0.0000  0.5516  0.5516  0.8097  0.3999
4  Queue2A   Class1  0.2935  0.2250  0.7251  0.3626 